In [2]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [5]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

## Warm-up Demo

In [7]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

CPU times: user 554 ms, sys: 6.49 ms, total: 560 ms
Wall time: 560 ms


In [9]:
%%time
for i,r in df.iterrows():
    r.X + r.Y

CPU times: user 341 ms, sys: 6.43 ms, total: 348 ms
Wall time: 348 ms


In [11]:
%%time
df.apply(lambda r: r.X + r.Y, axis = 1)

CPU times: user 129 ms, sys: 5.91 ms, total: 135 ms
Wall time: 133 ms


0       -38.083572
1       -38.171101
2       -38.091828
3       -38.066249
4       -38.062885
           ...    
27926   -38.176379
27927   -38.108333
27928   -38.073717
27929   -38.132576
27930   -38.123181
Length: 27931, dtype: float64

In [13]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

CPU times: user 400 μs, sys: 78 μs, total: 478 μs
Wall time: 433 μs


0       -38.083572
1       -38.171101
2       -38.091828
3       -38.066249
4       -38.062885
           ...    
27926   -38.176379
27927   -38.108333
27928   -38.073717
27929   -38.132576
27930   -38.123181
Length: 27931, dtype: float64

# Practice

In [15]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250']
C_Tar = C_Target = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']]
C_Oth = C_Other  = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

,CCN,WARD,OFFENSE,report_dt
6347,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00
17857,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00


**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [202]:
%%time

## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as shape
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head(20123250)

CPU times: user 9.75 ms, sys: 3.15 ms, total: 12.9 ms
Wall time: 10.1 ms


X          Y       CCN              REPORT_DAT  \
20165648 3826  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00   
         4348  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         5265  -76.994365  38.900203  20165859  2020/11/20 15:37:59+00   
         7027  -77.001316  38.898915  20165932  2020/11/20 18:56:18+00   
         7179  -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   
         7180  -76.997316  38.904969  20165803  2020/11/20 14:45:06+00   
         15446 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         15613 -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
20123250 3449  -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         4075  -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         21849 -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         22244 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         23901 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         27077 -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 3826        DAY  OTHERS         THEFT F/AUTO   
         4348   MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         5265        DAY  OTHERS          THEFT/OTHER   
         7027        DAY  OTHERS         THEFT F/AUTO   
         7179        DAY  OTHERS          THEFT/OTHER   
         7180        DAY  OTHERS         THEFT F/AUTO   
         15446   EVENING  OTHERS          THEFT/OTHER   
         15613   EVENING  OTHERS  MOTOR VEHICLE THEFT   
20123250 3449        DAY  OTHERS         THEFT F/AUTO   
         4075    EVENING  OTHERS  MOTOR VEHICLE THEFT   
         21849       DAY  OTHERS          THEFT/OTHER   
         22244       DAY  OTHERS          THEFT/OTHER   
         23901       DAY  OTHERS         THEFT F/AUTO   
         27077   EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                     BLOCK    XBLOCK  \
20165648 3826          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         4348             100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         5265               800 - 899 BLOCK OF H STREET NE  400489.0   
         7027               300 - 399 BLOCK OF G STREET NE  399886.0   
         7179    600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   
         7180          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         15446  300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         15613          1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
20123250 3449        2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0   
         4075             1700 - 1779 BLOCK OF M STREET NW  396546.0   
         21849            700 - 799 BLOCK OF 7TH STREET NW  398098.0   
         22244         1724 - 1799 BLOCK OF 17TH STREET NW  396662.0   
         23901            1700 - 1799 BLOCK OF P STREET NW  396523.0   
         27077            1100 - 1199 BLOCK OF F STREET NW  397609.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 3826   137456.0  ...       10602.0     Precinct 83  38.904961   
         4348   135959.0  ...        8200.0     Precinct 89  38.891475   
         5265   136927.0  ...        8402.0     Precinct 82  38.900195   
         7027   136784.0  ...        8301.0     Precinct 83  38.898907   
         7179   135255.0  ...        6500.0     Precinct 89  38.885133   
         7180   137456.0  ...       10602.0     Precinct 83  38.904961   
         15446  136899.0  ...        5900.0    Precinct 143  38.899942   
         15613  137478.0  ...       10603.0    Precinct 144  38.905159   
20123250 3449   138388.0  ...        4100.0     Precinct 13  38.913346   
         4075   137533.0  ...       10700.0     Precinct 17  38.905648   
         21849  136808.0  ...        5801.0    Precinct 129  38.899121   
         22244  138429.0  ...        5302.0     Precinct 15  38.913720   
         23901

# 1.5 Iterrow Approach

In [216]:
%%time

## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(r.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
#all_matches.head()
all_matches

CPU times: user 8.6 ms, sys: 4.4 ms, total: 13 ms
Wall time: 10.5 ms


X          Y       CCN              REPORT_DAT  \
20165648 3826  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00   
         4348  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         5265  -76.994365  38.900203  20165859  2020/11/20 15:37:59+00   
         7027  -77.001316  38.898915  20165932  2020/11/20 18:56:18+00   
         7179  -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   
         7180  -76.997316  38.904969  20165803  2020/11/20 14:45:06+00   
         15446 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         15613 -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
20123250 3449  -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         4075  -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         21849 -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         22244 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         23901 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         27077 -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 3826        DAY  OTHERS         THEFT F/AUTO   
         4348   MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         5265        DAY  OTHERS          THEFT/OTHER   
         7027        DAY  OTHERS         THEFT F/AUTO   
         7179        DAY  OTHERS          THEFT/OTHER   
         7180        DAY  OTHERS         THEFT F/AUTO   
         15446   EVENING  OTHERS          THEFT/OTHER   
         15613   EVENING  OTHERS  MOTOR VEHICLE THEFT   
20123250 3449        DAY  OTHERS         THEFT F/AUTO   
         4075    EVENING  OTHERS  MOTOR VEHICLE THEFT   
         21849       DAY  OTHERS          THEFT/OTHER   
         22244       DAY  OTHERS          THEFT/OTHER   
         23901       DAY  OTHERS         THEFT F/AUTO   
         27077   EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                     BLOCK    XBLOCK  \
20165648 3826          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         4348             100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         5265               800 - 899 BLOCK OF H STREET NE  400489.0   
         7027               300 - 399 BLOCK OF G STREET NE  399886.0   
         7179    600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   
         7180          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         15446  300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         15613          1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
20123250 3449        2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0   
         4075             1700 - 1779 BLOCK OF M STREET NW  396546.0   
         21849            700 - 799 BLOCK OF 7TH STREET NW  398098.0   
         22244         1724 - 1799 BLOCK OF 17TH STREET NW  396662.0   
         23901            1700 - 1799 BLOCK OF P STREET NW  396523.0   
         27077            1100 - 1199 BLOCK OF F STREET NW  397609.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 3826   137456.0  ...       10602.0     Precinct 83  38.904961   
         4348   135959.0  ...        8200.0     Precinct 89  38.891475   
         5265   136927.0  ...        8402.0     Precinct 82  38.900195   
         7027   136784.0  ...        8301.0     Precinct 83  38.898907   
         7179   135255.0  ...        6500.0     Precinct 89  38.885133   
         7180   137456.0  ...       10602.0     Precinct 83  38.904961   
         15446  136899.0  ...        5900.0    Precinct 143  38.899942   
         15613  137478.0  ...       10603.0    Precinct 144  38.905159   
20123250 3449   138388.0  ...        4100.0     Precinct 13  38.913346   
         4075   137533.0  ...       10700.0     Precinct 17  38.905648   
         21849  136808.0  ...        5801.0    Precinct 129  38.899121   
         22244  138429.0  ...        5302.0     Precinct 15  38.913720   
         23901

## 2. Function approach

Practice rewriting the above loop as a function

### 2.1 define the function

In [254]:
store_matches_2 = {}
def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    # Your code here
    
    
    ward_value = r.WARD
    ccn_vale = r.CCN
    report_dt_value = r.report_dt

    
    same_wards = C_Oth[C_Oth.WARD == ward_value]
    cutoff = report_dt_value + timedelta(minutes=1200)
    same_wards_sametime = same_wards[(same_wards.report_dt >= report_dt_value) & 
                                    (same_wards.report_dt <= cutoff)].copy()
    ## third, store the results
    store_matches_2[str(r.CCN)] = same_wards_sametime
    

### 2.2 apply it to one of the focal crimes

In [257]:
target_row = C_Tar.iloc[0]
find_related_crimes(target_row)
print(store_matches_2)

{'20165648':                X          Y       CCN              REPORT_DAT     SHIFT  \
3826  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00       DAY   
4348  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00  MIDNIGHT   
5265  -76.994365  38.900203  20165859  2020/11/20 15:37:59+00       DAY   
7027  -77.001316  38.898915  20165932  2020/11/20 18:56:18+00       DAY   
7179  -76.997328  38.885141  20165798  2020/11/20 12:46:32+00       DAY   
7180  -76.997316  38.904969  20165803  2020/11/20 14:45:06+00       DAY   
15446 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   EVENING   
15613 -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   EVENING   

       METHOD              OFFENSE  \
3826   OTHERS         THEFT F/AUTO   
4348   OTHERS  MOTOR VEHICLE THEFT   
5265   OTHERS          THEFT/OTHER   
7027   OTHERS         THEFT F/AUTO   
7179   OTHERS          THEFT/OTHER   
7180   OTHERS         THEFT F/AUTO   
15446  OTHERS          THEFT/OTHER   
15613  

### 2.3 Use apply to cover all the other focal crimes

In [260]:
%%time
C_Tar.apply(find_related_crimes, axis=1)

CPU times: user 7.96 ms, sys: 2.82 ms, total: 10.8 ms
Wall time: 8.15 ms


6347     None
17857    None
dtype: object

In [262]:
pd.concat(store_matches_2)

X          Y       CCN              REPORT_DAT  \
20165648 3826  -76.997316  38.904969  20165805  2020/11/20 15:06:04+00   
         4348  -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         5265  -76.994365  38.900203  20165859  2020/11/20 15:37:59+00   
         7027  -77.001316  38.898915  20165932  2020/11/20 18:56:18+00   
         7179  -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   
         7180  -76.997316  38.904969  20165803  2020/11/20 14:45:06+00   
         15446 -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         15613 -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
20123250 3449  -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         4075  -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         21849 -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         22244 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         23901 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         27077 -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 3826        DAY  OTHERS         THEFT F/AUTO   
         4348   MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         5265        DAY  OTHERS          THEFT/OTHER   
         7027        DAY  OTHERS         THEFT F/AUTO   
         7179        DAY  OTHERS          THEFT/OTHER   
         7180        DAY  OTHERS         THEFT F/AUTO   
         15446   EVENING  OTHERS          THEFT/OTHER   
         15613   EVENING  OTHERS  MOTOR VEHICLE THEFT   
20123250 3449        DAY  OTHERS         THEFT F/AUTO   
         4075    EVENING  OTHERS  MOTOR VEHICLE THEFT   
         21849       DAY  OTHERS          THEFT/OTHER   
         22244       DAY  OTHERS          THEFT/OTHER   
         23901       DAY  OTHERS         THEFT F/AUTO   
         27077   EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                     BLOCK    XBLOCK  \
20165648 3826          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         4348             100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         5265               800 - 899 BLOCK OF H STREET NE  400489.0   
         7027               300 - 399 BLOCK OF G STREET NE  399886.0   
         7179    600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   
         7180          600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         15446  300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         15613          1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
20123250 3449        2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0   
         4075             1700 - 1779 BLOCK OF M STREET NW  396546.0   
         21849            700 - 799 BLOCK OF 7TH STREET NW  398098.0   
         22244         1724 - 1799 BLOCK OF 17TH STREET NW  396662.0   
         23901            1700 - 1799 BLOCK OF P STREET NW  396523.0   
         27077            1100 - 1199 BLOCK OF F STREET NW  397609.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 3826   137456.0  ...       10602.0     Precinct 83  38.904961   
         4348   135959.0  ...        8200.0     Precinct 89  38.891475   
         5265   136927.0  ...        8402.0     Precinct 82  38.900195   
         7027   136784.0  ...        8301.0     Precinct 83  38.898907   
         7179   135255.0  ...        6500.0     Precinct 89  38.885133   
         7180   137456.0  ...       10602.0     Precinct 83  38.904961   
         15446  136899.0  ...        5900.0    Precinct 143  38.899942   
         15613  137478.0  ...       10603.0    Precinct 144  38.905159   
20123250 3449   138388.0  ...        4100.0     Precinct 13  38.913346   
         4075   137533.0  ...       10700.0     Precinct 17  38.905648   
         21849  136808.0  ...        5801.0    Precinct 129  38.899121   
         22244  138429.0  ...        5302.0     Precinct 15  38.913720   
         23901